In [3]:
import pandas as pd
import geopandas as gpd


# Loading Geodataframes for crime data

In [4]:
# Major Crime Indicator Data

shapefile_path_crime = 'Major_Crime_Indicators_Open_Data'

# Read the shapefile (this will automatically read the associated .dbf file)
crime_points = gpd.read_file(shapefile_path_crime + '.shp')


In [5]:
# Shooting Data

# Replace 'your_shapefile.shp' with the actual path to your shapefile (without the extension)
shapefile_path_shoot = 'Shooting_and_Firearm_Discharges_Open_Data'

# Read the shapefile (this will automatically read the associated .dbf file)
shooting = gpd.read_file(shapefile_path_shoot + '.shp')

In [6]:
# Homicide Data

# Replace 'your_shapefile.shp' with the actual path to your shapefile (without the extension)
shapefile_path_hom = 'Homicides_Open_Data_(ASR-RC-TBL-002)'

# Read the shapefile (this will automatically read the associated .dbf file)
homicide = gpd.read_file(shapefile_path_hom + '.shp')

# Filtering Crime Data to Street Safety Related Crimes

In [17]:
crime_points = crime_points[crime_points["MCI_CATEGO"].isin(["Assault","Robbery"])]

# Joining Homicide and Shooting Data with Crimes

## Checking which columns are common amongst all dataframes

In [34]:
list(set(crime_points.columns).intersection(homicide.columns, shooting.columns))

['NEIGHBOU_1',
 'OCC_YEAR',
 'HOOD_140',
 'OCC_DOW',
 'DIVISION',
 'EVENT_UNIQ',
 'geometry',
 'OCC_DAY',
 'OCC_DATE',
 'LAT_WGS84',
 'LONG_WGS84',
 'OCC_DOY',
 'NEIGHBOURH',
 'HOOD_158',
 'OCC_MONTH',
 'OBJECTID']

In [35]:
crime_join = crime_points[list(set(crime_points.columns).intersection(homicide.columns, shooting.columns))]
hom_join = homicide[list(set(crime_points.columns).intersection(homicide.columns, shooting.columns))]
shoot_join = shooting[list(set(crime_points.columns).intersection(homicide.columns, shooting.columns))]

crimes = pd.concat([crime_join, hom_join, shoot_join], ignore_index=True)

In [36]:
crimes

,NEIGHBOU_1,OCC_YEAR,HOOD_140,OCC_DOW,DIVISION,EVENT_UNIQ,geometry,OCC_DAY,OCC_DATE,LAT_WGS84,LONG_WGS84,OCC_DOY,NEIGHBOURH,HOOD_158,OCC_MONTH,OBJECTID
0,Rosedale-Moore Park (98),2014.0,98,Wednesday,D53,GO-20141265238,POINT (-8837009.381 5414637.989),1.0,2014-01-01,43.670798,-79.384206,1.0,Rosedale-Moore Park,98,January,1
1,Thorncliffe Park (55),2014.0,55,Wednesday,D53,GO-20141259834,POINT (-8832733.476 5419700.571),1.0,2014-01-01,43.703684,-79.345795,1.0,Thorncliffe Park,55,January,2
2,Waterfront Communities-The Island (77),2014.0,77,Wednesday,D52,GO-20141262027,POINT (-8836444.480 5410819.359),1.0,2014-01-01,43.645981,-79.379131,1.0,St Lawrence-East Bayfront-The Islands,166,January,3
3,Bay Street Corridor (76),2014.0,76,Wednesday,D52,GO-20141259951,POINT (-8836897.420 5412101.189),1.0,2014-01-01,43.654313,-79.383200,1.0,Yonge-Bay Corridor,170,January,4
4,Downsview-Roding-CFB (26),2014.0,26,Wednesday,D31,GO-20141261561,POINT (-8851435.343 5422186.195),1.0,2014-01-01,43.719824,-79.513797,1.0,Oakdale-Beverley Heights,154,January,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232208,Kensington-Chinatown (78),2023.0,78,Saturday,D14,GO-2023602110,POINT (-8838833.625 5411111.786),18.0,2023-03-18,43.647882,-79.400593,77.0,Kensington-Chinatown,78,March,5946
232209,Blake-Jones (69),2023.0,69,Friday,D55,GO-20231811181,POINT (-8832025.174 5416055.479),4.0,2023-08-04,43.680008,-79.339432,216.0,Blake-Jones,69,August,5947
232210,Rockcliffe-Smythe (111),2023.0,111,Friday,D12,GO-20231873884,POINT (-8847820.740 5416594.779),11.0,2023-08-11,43.683512,-79.481326,223.0,Rockcliffe-Smythe,111,August,5948
232211,West Hill (136),2023.0,136,Saturday,D43,GO-20232284577,POINT (-8814969.936 5429671.668),30.0,2023-09-30,43.768403,-79.186222,273.0,West Hill,136,September,5949


# Clustering the full crime dataset using DBSCAN

## Checking the distribution of latlongs to remove blanks

In [37]:
crimes.LAT_WGS84.value_counts()

LAT_WGS84
0.000000     3290
43.612078    1642
43.656323     975
43.654057     612
43.778438     579
             ... 
43.673420       1
43.696733       1
43.645045       1
43.700847       1
43.812723       1
Name: count, Length: 18581, dtype: int64

Seems like there is a small proportion of blank latlongs

In [40]:
crimes = crimes[crimes["LAT_WGS84"]!=0]

## Display Crimes

In [42]:
pip install folium


  Obtaining dependency information for folium from https://files.pythonhosted.org/packages/18/09/8569904c8ce5679cc02826d98de633c07abcd2443a23181e5f71ff9dacbc/folium-0.15.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for branca>=0.6.0 from https://files.pythonhosted.org/packages/2f/e7/603b136221de923055716d23e3047da71f92e0d8ba2c4517ce49a54fe768/branca-0.7.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/97.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/97.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/97.0 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/97.0 kB ? eta -:--:--
   ------------ --------------------------- 30.7/97.0 kB 330.3 kB/s eta 0:00:01
   ---------------- ----------------------- 41.0/97.0 kB 330.3 kB/s eta 0:00:01
   ------------------------------------- -- 92.2/97.0 kB 525.1 kB/s eta 0:00:01
   ---------------------------------------- 97.0/97.0 kB 507.6 kB/s eta 

In [47]:
crimes.OCC_YEAR.value_counts()

OCC_YEAR
2019.0    24786
2022.0    24016
2018.0    23918
2017.0    23848
2016.0    23029
2015.0    21851
2021.0    21482
2020.0    21402
2014.0    20503
2023.0    20011
2013.0      676
2012.0      459
2010.0      403
2011.0      394
2009.0      379
2005.0      363
2008.0      347
2007.0      319
2006.0      300
2004.0      275
2000.0       19
2001.0       19
2002.0       16
2003.0       16
Name: count, dtype: int64

In [48]:
crimes_5year = crimes[crimes["OCC_YEAR"]>2017]

In [51]:
crimes_5year = crimes_5year.reset_index()

## Performing Clustering

In [ ]:
def haversine_distance(lon1, lat1, lon2, lat2,**kwargs):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    All args must be of equal length.
    
    Sourced from https://stackoverflow.com/a/29546836/11637704
    
    Thanks to derricw!
    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [55]:
from sklearn.cluster import DBSCAN
import numpy as np

def dbscan_cluster(latitudes,longitudes,epsilon,min_samples,**kwargs):
    '''
    Function to perform DBSCAN clustering for given parameters.
    
    '''
    
    # convert epsilon from km to radians
    kms_per_radian = 6371.0088
    epsilon /= kms_per_radian
    
    # set up the algorithm
    dbscan = DBSCAN(
        eps = epsilon,
        min_samples = min_samples,
        algorithm = 'ball_tree',
        metric = 'haversine',
        **kwargs
    )
    
    # fit the algorithm
    dbscan.fit(
        np.radians(
            [x for x in zip(latitudes,longitudes)]
        )
    )
    
    # return the cluster labels
    return pd.Series(dbscan.labels_)

In [ ]:
def vertex_centroid_distance(latitudes,longitudes,**kwargs):
    '''
    Function to calculate the average distance from the vertices of a convex hull
    (derived from latitude x longitude pairs) to the centroid of said convex hull.
    
    Centroid is taken to be the unweighted average of all co-ordinate pairs.
    
    '''
    
    # co-ordinates of centre
    # take a simple average
    centre_long = longitudes.mean()
    centre_lats = latitudes.mean()
    
    # collapse two points into line
    if len(latitudes) < 3:
        distances = haversine_distance(
            longitudes,
            latitudes,
            centre_long,
            centre_lats,
            **kwargs).mean()
    
    else:
        # convex hull
        convex_hull = ConvexHull([x for x in zip(latitudes,longitudes)],**kwargs)

        # now get co-ordinates of vertices
        vertex_longs = longitudes.iloc[convex_hull.vertices]
        vertex_lats = latitudes.iloc[convex_hull.vertices]

        # now get
        distances = haversine_distance(
            vertex_longs,
            vertex_lats,
            centre_long,
            centre_lats,
            **kwargs).mean()

    # return average distance
    return distances.mean()

In [58]:
dbscan_cluster(crimes[crimes["OCC_YEAR"]>2020]["LAT_WGS84"],crimes[crimes["OCC_YEAR"]>2020]["LONG_WGS84"],5,8)

0        0
1        0
2        0
3        0
4        0
        ..
65504    0
65505    0
65506    0
65507    0
65508    0
Length: 65509, dtype: int64

In [59]:
from sklearn.cluster import KMeans

In [61]:
kms_per_radian = 6371.0088

In [65]:
# import necessary modules
import pandas as pd, numpy as np, matplotlib.pyplot as plt, time
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
from datetime import datetime as dt

# magic command to display matplotlib plots inline within the ipython notebook
%matplotlib inline

In [69]:
df_crime = pd.DataFrame(crimes.drop(columns='geometry'))

In [72]:
df_crime.as_matrix(columns=["LAT_WGS84", "LONG_WGS84"])

AttributeError: 'DataFrame' object has no attribute 'as_matrix'

In [66]:
from sklearn.cluster import DBSCAN
import numpy as np

def dbscan_reduce(df, epsilon, x='LONG_WGS84', y='LAT_WGS84'):
    start_time = time.time()
    # represent points consistently as (lat, lon) and convert to radians to fit using haversine metric
    coords = df.as_matrix(columns=["LAT_WGS84", "LONG_WGS84"])    
    db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
    cluster_labels = db.labels_
    num_clusters = len(set(cluster_labels))
    print('Number of clusters: {:,}'.format(num_clusters))
    
    clusters = pd.Series([coords[cluster_labels==n] for n in range(num_clusters)])
    
    # find the point in each cluster that is closest to its centroid
    centermost_points = clusters.map(get_centermost_point)

    # unzip the list of centermost points (lat, lon) tuples into separate lat and lon lists
    lats, lons = zip(*centermost_points)
    rep_points = pd.DataFrame({x:lons, y:lats})
    rep_points.tail()
    
    # pull row from original data set where lat/lon match the lat/lon of each row of representative points
    rs = rep_points.apply(lambda row: df[(df[y]==row[y]) & (df[x]==row[x])].iloc[0], axis=1)
    
    # all done, print outcome
    message = 'Clustered {:,} points down to {:,} points, for {:.2f}% compression in {:,.2f} seconds.'
    print(message.format(len(df), len(rs), 100*(1 - float(len(rs)) / len(df)), time.time()-start_time))    
    return rs

In [70]:

df_clustered = dbscan_reduce(df_crime[df_crime["OCC_YEAR"]>2020], epsilon=eps_rad)

AttributeError: 'DataFrame' object has no attribute 'as_matrix'

In [74]:
coords = df_crime[df_crime["OCC_YEAR"]>2020][["LAT_WGS84","LONG_WGS84"]].to_numpy()   


In [98]:
eps_rad = 10 / 6371.0088

In [86]:
db = DBSCAN(eps=eps_rad, min_samples=10, algorithm='ball_tree', metric='haversine').fit_predict(np.radians(coords))

In [99]:
db_1 = DBSCAN(eps=eps_rad, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))

MemoryError: bad allocation

In [ ]:
db_1.labels_

In [87]:
np.unique(db)

array([-1,  0,  1,  2], dtype=int64)

In [91]:
labels = db

In [88]:
df = df_crime.copy()

In [89]:
df = df[["LAT_WGS84","LONG_WGS84"]]

In [92]:
# Visualising the clusters
plt.scatter(df[labels == -1, 0], df[labels == -1, 1], s = 10, c = 'black') 

plt.scatter(df[labels == 0, 0], df[labels == 0, 1], s = 10, c = 'blue')
plt.scatter(df[labels == 1, 0], df[labels == 1, 1], s = 10, c = 'red')
plt.scatter(df[labels == 2, 0], df[labels == 2, 1], s = 10, c = 'green')
plt.scatter(df[labels == 3, 0], df[labels == 3, 1], s = 10, c = 'brown')
plt.scatter(df[labels == 4, 0], df[labels == 4, 1], s = 10, c = 'pink')
plt.scatter(df[labels == 5, 0], df[labels == 5, 1], s = 10, c = 'yellow')      
plt.scatter(df[labels == 6, 0], df[labels == 6, 1], s = 10, c = 'silver')

plt.xlabel('Annual Income')
plt.ylabel('Spending Score')
plt.show()

InvalidIndexError: (array([False, False, False, ..., False, False, False]), 0)

In [78]:
dir(db)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_request_for_signature',
 '_check_feature_names',
 '_check_n_features',
 '_estimator_type',
 '_get_default_requests',
 '_get_metadata_request',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_validate_data',
 '_validate_params',
 'algorithm',
 'components_',
 'core_sample_indices_',
 'eps',
 'fit',
 'fit_predict',
 'get_metadata_routing',
 'get_params',
 'labels_',
 'leaf_size',
 'metric',
 'metric_params',
 'min_samples',
 'n_features_in_',
 'n_jobs'

In [81]:
db.core_sample_indices_

array([    0,     1,     2, ..., 65506, 65507, 65508], dtype=int64)